In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import parabolic
from numpy.fft import rfft, irfft
from numpy import argmax, sqrt, mean, absolute, linspace, log10, logical_and, average, diff, correlate
from scipy.signal import blackmanharris, fftconvolve
import time
import sys

None

In [2]:
from __future__ import division

def parabolic(f, x):
    """Quadratic interpolation for estimating the true position of an
    inter-sample maximum when nearby samples are known.
   
    f is a vector and x is an index for that vector.
   
    Returns (vx, vy), the coordinates of the vertex of a parabola that goes
    through point x and its two neighbors.
   
    Example:
    Defining a vector f with a local maximum at index 3 (= 6), find local
    maximum if points 2, 3, and 4 actually defined a parabola.
   
    In [3]: f = [2, 3, 1, 6, 4, 2, 3, 1]
   
    In [4]: parabolic(f, argmax(f))
    Out[4]: (3.2142857142857144, 6.1607142857142856)
   
    """
    # Requires real division.  Insert float() somewhere to force it?
    xv = 1/2 * (f[x-1] - f[x+1]) / (f[x-1] - 2 * f[x] + f[x+1]) + x
    yv = f[x] - 1/4 * (f[x-1] - f[x+1]) * (xv - x)
    return (xv, yv)

In [3]:
def freq_from_autocorr(sig, fs):
    """Estimate frequency using autocorrelation
    
    Pros: Best method for finding the true fundamental of any repeating wave, 
    even with strong harmonics or completely missing fundamental
    
    Cons: Not as accurate, currently has trouble with finding the true peak
    
    """
    # Calculate autocorrelation and throw away the negative lags
    corr = fftconvolve(sig, sig[::-1], mode='full')
    corr = corr[len(corr)//2:]
    
    # Find the first low point
    d = diff(corr)
    # Find the index where the difference is greater than 0
    start = np.where(d > 0)[0][0] if np.any(d > 0) else None
    
    # Find the next peak after the low point (other than 0 lag).  This bit is 
    # not reliable, due to peaks that occur between samples.
    peak = argmax(corr[start:]) + start
    px,py= parabolic(corr, peak)
    
    return fs / px

In [4]:
excel_file="sinwave_snr10dB_1.5Hz_Fs100Hz.xlsx"
df=pd.read_excel(excel_file)
sig= df['amplitude'].values
fs=100
estimated_frequency = freq_from_autocorr(sig,fs)
print(estimated_frequency)




KeyError: 'amplitude'